In [9]:
import litellm
from litellm import completion
import numpy as np
litellm.drop_params = True

def call_llm(prompt, **kwargs):
    if "temperature" not in kwargs:
        kwargs["temperature"] = 1.2
    response = completion(
        model="ollama/llama3.1", 
        messages=[{ "content": prompt,"role": "user"}], 
        api_base="http://localhost:11434",
        **kwargs
    )
    choice = response.choices[0]
    if choice.finish_reason == "tool_calls":
        tool_call_data = choice.message.tool_calls[0]
        tool_call_str = f"{tool_call_data.function.name}(**{tool_call_data.function.arguments})"
        return eval(tool_call_str)
    else:
        return choice.message.content

In [38]:
reviews = [
    "I loved this movie!",
    "I hated this movie!",
    "I watched this movie.",
]

In [47]:
zero_shot_sentiment_template = """
Classify the sentiment of the following text as positive, negative, or neutral. \
Only return the classification. 
Text: {text}
Answer: \
"""
for review in reviews:
    print(review)
    print(call_llm(zero_shot_sentiment_template.format(text=review)))

I loved this movie!
Positive
I hated this movie!
Negative
I watched this movie.
Neutral


In [48]:
few_shot_sentiment_template = """
Classify the sentiment of the following text as positive, negative, or neutral. \
Only return the classification. \
Examples:

Text: The food was delicious
Answer: positive

Text: I did not enjoy the scenery
Answer: negative

Text: Go straight then turn left at the stop sign
Answer: neutral

Text: It smelled so bad
Answer: negative

Text: I had a great time
Answer: positive

Text: {text}
Answer: \
"""
for review in reviews:
    print(review)
    print(call_llm(few_shot_sentiment_template.format(text=review)))

I loved this movie!
positive
I hated this movie!
negative
I watched this movie.
neutral


In [24]:
for _ in range(3):
    print(call_llm("What is (1+3)^2 - 2? Let's think step by step.", temperature=1.99))
    print("=" * 50)

Let's break it down step by step.

Step 1: Evaluate the expression inside the parentheses, (1+3).

(1+3) = 4

So, now we have: 4^2 - 2

Step 2: Evaluate the exponentiation, 4^2.

4^2 = 16

Now we have: 16 - 2

Step 3: Finally, subtract 2 from 16.

16 - 2 = 14

So, (1+3)^2 - 2 = 14.
Let's break down the expression step by step.

We start with the inside of the parentheses:

(1 + 3) = 4

So now we have:

4^2 - 2

Next, let's evaluate the exponent:

4^2 = 16 (since 4 multiplied by itself is 16)

Now we have:

16 - 2

Finally, let's subtract:

16 - 2 = 14

Therefore, the final answer is:

14
To find the value of (1+3)^2 - 2, let's follow the order of operations.

**Step 1: Evaluate the expression inside the parentheses**
(1+3) = 4

**Step 2: Square the result**
4^2 = 16

**Step 3: Subtract 2 from the squared result**
16 - 2 = 14

Therefore, (1+3)^2 - 2 = 14.


In [15]:
def get_current_weather(city):
    weather = np.random.choice(["sunny", "cloudy", "rainy", "snowy"])
    return f"The weather in {city} is {weather}."

In [16]:
tools=[{
      'type': 'function',
      'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    },
  ],

In [25]:
for _ in range(5):
    print(call_llm("What is the weather in Paris?", tools=tools))

The weather in Paris is snowy.
The weather in Paris is sunny.
The weather in Paris is cloudy.
The weather in Paris is rainy.
The weather in Paris is snowy.
